# Proyecto práctico: árbol de decisión y random forest con scikit-learn

In [1]:
#Importamos las librerias principales
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
from pprint import pprint as pp

Utilizaremos el **Car Evaluation Data Set** de Kaggle: https://www.kaggle.com/datasets/elikplim/car-evaluation-data-set

In [2]:

path="/home/dan/PLATZI/data/Fundamentals/randomforestsalgorithms/data/external"
colum_names=["buying price","maintinance price","number of doors","persons capacity","luggage boot","safty stimated","target"]

#Cargamos dataset a utilizar
df=pd.read_csv(path+"/car_evaluation.csv")
df.columns=colum_names
categorical= df.select_dtypes(include=['object']).columns
uniqueValsSumary={
    col:{'unique count':df[col].nunique(),'unique values':df[col].unique().tolist()}
    for col in categorical
    
}
uniqueValsSumary=pd.DataFrame.from_dict(uniqueValsSumary,orient='index')
uniqueValsSumary


,unique count,unique values
buying price,4,"[vhigh, high, med, low]"
maintinance price,4,"[vhigh, high, med, low]"
number of doors,4,"[2, 3, 4, 5more]"
persons capacity,3,"[2, 4, more]"
luggage boot,3,"[small, med, big]"
safty stimated,3,"[med, high, low]"
target,4,"[unacc, acc, vgood, good]"


it apear all columns are categorical  by default even the ones that use numbers, a proper encoding will have to be in order

## Análisis exploratorio de datos

In [3]:
#Visualizacion del dataframe
df.head(3)

,buying price,maintinance price,number of doors,persons capacity,luggage boot,safty stimated,target
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc


In [4]:
#Analizamos el shape del objeto
print("size of the data frame")
print(df.size)
print("="*64)
print("frecuenzy of each category")
for col in df.columns:
    print(f"Value counts for: {col}")
    print(df[col].value_counts())
    print("\n")


size of the data frame
12089
frecuenzy of each category
Value counts for: buying price
buying price
high     432
med      432
low      432
vhigh    431
Name: count, dtype: int64


Value counts for: maintinance price
maintinance price
high     432
med      432
low      432
vhigh    431
Name: count, dtype: int64


Value counts for: number of doors
number of doors
3        432
4        432
5more    432
2        431
Name: count, dtype: int64


Value counts for: persons capacity
persons capacity
4       576
more    576
2       575
Name: count, dtype: int64


Value counts for: luggage boot
luggage boot
med      576
big      576
small    575
Name: count, dtype: int64


Value counts for: safty stimated
safty stimated
med     576
high    576
low     575
Name: count, dtype: int64


Value counts for: target
target
unacc    1209
acc       384
good       69
vgood      65
Name: count, dtype: int64




the target  classes are unbalance ofcourse here the numbers
- unacc    1209
- acc       384
- good       69
- vgood      65
all the other columns are more less at equilibrium so the distributions are uniform, there are no missing values .

In [ ]:
#we will separate and train a class tree not having on acount the umbalance
X=df.drop(["target"],axis=1)
Y=df["target"]
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3, random_state=37,stratify=Y)
# the stratify astribute is necesary because the  encoder for labels does not know what to doo 
# when  in the test set you ask it to transforma a category never seen

In [28]:
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.preprocessing import LabelEncoder
encoder=OrdinalEncoder()
labeler=LabelEncoder()
X_train=encoder.fit_transform(X_train)
X_test=encoder.transform(X_test)
Y_train=labeler.fit_transform(Y_train)
Y_test=labeler.transform(Y_test)
from sklearn.tree  import DecisionTreeClassifier
tree=DecisionTreeClassifier(max_depth=2,random_state=37)
tree.fit(X_train,Y_train)
train_predictions=tree.predict(X_train)
predictions=tree.predict(X_test)
print(tree.feature_importances_)

[0.         0.         0.         0.60357498 0.         0.39642502]


In [12]:
from sklearn.metrics import accuracy_score
#cheking for over fitting
train_accuracy=accuracy_score(Y_train,train_predictions)
accuracy=accuracy_score(Y_test,predictions)
print(f"train accuracy:{np.round(train_accuracy,2)}, and test accuracy: {np.round(accuracy,2)}")


train accuracy:0.76, and test accuracy: 0.78


as you see the accuracy in both scenarios is simillar so  there is no over or under fitting, never the lees the accuracy should be  o between 90 and 80 porcent to have a good model., lets traina model with a more balance data set to see what happens


In [15]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=37)
# ros dont work easyly with no numeric features.
X_resampled, y_resampled = ros.fit_resample(X_train, Y_train)
#stratified sampling  ensures the train test split respects the original dataset ratio
balancedTree=DecisionTreeClassifier(max_depth=2,random_state=37)
balancedTree.fit(X_train,Y_train)
train_predictions=balancedTree.predict(X_train)
predictions= balancedTree.predict(X_test)
#accuracy mesures
train_accuracy=accuracy_score(Y_train,train_predictions)
accuracy=accuracy_score(Y_test,predictions)
print(f"train accuracy:{np.round(train_accuracy,2)}, and test accuracy: {np.round(accuracy,2)}")

train accuracy:0.76, and test accuracy: 0.78


### This is a surprise

When the hyperparameter `class_weight` is used in `'balanced'` mode, what happens under the hood is that an error misclassifying a minority class gets a higher penalty. This can lead to **overfitting** for the minority class unless parameters like `max_depth`, `min_samples_leaf`, or `min_samples_split` are properly tuned.

Also, plain accuracy may not be the best metric because of this overfitting. A model might perform very well on the majority class and poorly on the minority class, yet still show high accuracy. This can be misleading — especially if, in production, the class imbalance does not exist.

In other cases, manual balancing is necessary using techniques like **SMOTE**, **random oversampling**, or **undersampling**.


#### balanced accuracy score:

In [16]:
#balanced accuracy makes a  weigthed avrage of the recall
from sklearn.metrics import balanced_accuracy_score

train_accuracy=balanced_accuracy_score(Y_train,train_predictions)
accuracy=balanced_accuracy_score(Y_test,predictions)
print(f"train accuracy:{np.round(train_accuracy,2)}, and test accuracy: {np.round(accuracy,2)}")


train accuracy:0.36, and test accuracy: 0.37


So, our decision tree is actually worse than we thought. Let's try to manually balance the dataset and create another tree to see what happens.

Since our minority class has around **50 data points**, and decision trees tend to work well with small datasets, I feel inclined to use **undersampling**. However, the final decision will depend on **IBM's `imblearn` compatibility**.


In [22]:
from imblearn.under_sampling import RandomUnderSampler
print("before undersampling",Y.value_counts())
undersampler=RandomUnderSampler(random_state=37)
x,y=undersampler.fit_resample(X,Y)
print("after undersampling",y.value_counts())
encoder=OrdinalEncoder()
labeler=LabelEncoder()
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=37,stratify=y)
x_train=encoder.fit_transform(x_train)
x_test=encoder.transform(x_test)
y_train=labeler.fit_transform(y_train)
y_test=labeler.transform(y_test)
tree=DecisionTreeClassifier(max_depth=2,random_state=37)
tree=tree.fit(x_train,y_train)


before undersampling target
unacc    1209
acc       384
good       69
vgood      65
Name: count, dtype: int64
after undersampling target
acc      65
good     65
unacc    65
vgood    65
Name: count, dtype: int64


In [ ]:
train_predictions=tree.predict(x_train)
predictions=tree.predict(x_test)
accuracy_train=accuracy_score(y_train,train_predictions)
accuracy_test=accuracy_score(y_test,predictions)
print(f"train accuracy {accuracy_train}, test accuracy{accuracy_test}")
#when no maxdepth use  the results are 1 and 0.88 a very likely overfitting

train accuracy 0.5274725274725275, test accuracy0.5384615384615384


no better than a coin toss , meyby with  more samples

In [25]:
from imblearn.over_sampling import RandomOverSampler
print("before undersampling",Y.value_counts())
oversampler=RandomOverSampler(random_state=37)
x,y=oversampler.fit_resample(X,Y)
print("after undersampling",y.value_counts())
encoder=OrdinalEncoder()
labeler=LabelEncoder()
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=37,stratify=y)
x_train=encoder.fit_transform(x_train)
x_test=encoder.transform(x_test)
y_train=labeler.fit_transform(y_train)
y_test=labeler.transform(y_test)
tree=DecisionTreeClassifier(max_depth=2,random_state=37)
tree=tree.fit(x_train,y_train)



before undersampling target
unacc    1209
acc       384
good       69
vgood      65
Name: count, dtype: int64
after undersampling target
unacc    1209
acc      1209
vgood    1209
good     1209
Name: count, dtype: int64


In [26]:
train_predictions=tree.predict(x_train)
predictions=tree.predict(x_test)
accuracy_train=accuracy_score(y_train,train_predictions)
accuracy_test=accuracy_score(y_test,predictions)
print(f"train accuracy {accuracy_train}, test accuracy{accuracy_test}")

train accuracy 0.507533234859675, test accuracy0.5134390075809786


what happens here  having on acount that when the data set wasa imbalanced, in proportions was like having just 2 clases, when its  balanced we do have 4 classes it makes me think, decicion trees are very good in binary clasification, but not in multy class. lets see the feature importance in the   balanced tree vs the umbalance tree


In [27]:
features=tree.feature_importances_
print(features)

[0.         0.         0.         0.56517124 0.         0.43482876]


In [ ]:
#Veamos que tenemos. Por ejemplo, en X_train
    

## Entrenamiento de modelo de clasificación con árbol de decisión

In [ ]:
#Importante: todos nuestros tipos de datos son object, realizamos una transformacion


In [ ]:
#Verificamos la transformacion


In [ ]:
#Importar árbol de decisión

#Creacion del modelo


In [ ]:
#Entrenamiento


In [ ]:
#Calculo de las predicciones en Train y Test


## Evaluación de modelo de clasificación con árbol de decisión

In [ ]:
#Calculo de metricas 


#Calculo el accuracy en Train


#Calculo el accuracy en Test


In [ ]:
#Verificamos el feature importances


## Entrenamiento de modelo de clasificación con random forest

In [ ]:
#Importar random forest


In [ ]:
#Calculo de las predicciones en Train y Test


## Evaluación de modelo de clasificación con random forest

In [ ]:
#Calculo de metricas 


#Calculo el accuracy en Train


#Calculo el accuracy en Test


#Importante: podriamos reducir el numero de estimadores para disminuir el sobreajuste del modelo.

In [ ]:
# Visualizacion de las feature importantes


In [ ]:
#Grafico de barras


In [ ]:
# Matriz de confusion del RF


In [ ]:
#RF
